# Self-Consistency and Chain-of-thoughts prompting in LLMs

In this notebook, I'll employ the [Mixtral8x7B-Instruct](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1) LLM model to explore and apply chain-of-thought and self-consistency prompting techniques. This involves practically learning these methods by applying on various examples.

This  LLM is capable enough to run on google colab thanks to **Denis Mazur** and **Artyom Eliseev** who have made this possible by quantizing the original model in mixed precision and implementing a MoE-specific offloading strategy. `[Quantization, build_model, Expert MoE files were forked from their github repo]`.

Read their [tech report](https://arxiv.org/abs/2312.17238). I will try to summarize my learnings from their report in the following days in this notebook.


##### *Edited: 01-09-2024.*

## Install and import libraries

In [1]:
from IPython.display import HTML, display, Markdown

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

HTML('''<script>
code_show_err=false;
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
}
$( document ).ready(code_toggle_err);
</script>''')

In [2]:
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/emmanuelrajapandian/Advanced-Prompt-Engineering-LLMs.git --quiet
!cd Advanced-Prompt-Engineering-LLMs && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

In [3]:
import sys

sys.path.append("Advanced-Prompt-Engineering-LLMs")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from langchain.prompts import PromptTemplate
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

from source.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Initialize model

In [4]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)
device = torch.device("cuda:0")

offload_per_layer = 4
num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)

attn_config["scale_quant_params"]["group_size"] = 256

ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


# Building the Model after specifying the params in the instantiated Classes
# OffloadConfig(), BaseQuantizeConfig().
model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

clear_output()

## Run the model

In [5]:
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

clear_output()

In [6]:
def generate_outputs(prompt:str, temperature:float, max_tokens:int=128, do_sample:bool=True, stop_sequences:list=[], n:int=1) -> list:
    """
    Function to run inference with Mixtral model
    """
    user_entry = dict(role="user", content=prompt)
    input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

    outputs = []
    for i in range(n):
        outputs.append(model.generate(input_ids=input_ids,
                                      streamer=streamer,
                                      do_sample=do_sample,
                                      temperature=temperature,
                                      top_p=0.9,
                                      max_new_tokens=max_tokens,
                                      pad_token_id=tokenizer.eos_token_id,
                                      return_dict_in_generate=True,
                                      output_hidden_states=False,))
    return outputs

In [ ]:
TEMP = 0.9
output = generate_outputs("What is the pet of Phineas and Ferb called and what does it do?", TEMP, max_tokens=128)

Phineas Flynn and Ferb Fletcher's pet in the animated show "Phineas and Ferb" is a platypus named Perry. However, Perry is not just an ordinary pet, as he leads a double life: when the boys are not using him for a backyard adventure, Perry removes his pet collar and becomes Agent P, a secret agent fighting evil.

Perry, the pet platypus, does not have any supernatural powers, but he is very intelligent, agile, and skilled in martial arts. He uses gadgets and tools created by Phineas


In [ ]:
TEMP = 0.9
output = generate_outputs("What are the commands in Mac to push, pull, merge repos in github from the local terminal?",
                          TEMP, max_tokens=256)

To perform Git operations on a Mac, you will first need to navigate to the local directory that contains the Git repository using the `cd` command. Once you are in the directory that contains the Git repository, you can use the following commands:

* To pull changes from the remote repository, use the command:
```
git pull origin <branch-name>
```
* To push your local changes to the remote repository, use the command:
```
git push origin <branch-name>
```
* To merge changes from one branch onto another, use the command:
```
git merge branch-name
```
Here, `origin` refers to the remote repository where the Git operations are being performed, and `<branch-name>` is the name of the branch that you want to merge.

You may also need to use the `git add` and `git commit` commands to add and commit changes to your local repository before pushing them to the remote repository.

To learn more about these and other Git commands, you can refer to the official Git documentation.


Greedy Decoding

In [ ]:
prompt = "In less than 40 words, finish the sentence 'I want to go to the market to"
TEMP = 0.0

while True:
  output = generate_outputs(prompt, TEMP, do_sample=False, max_tokens=64, n=3)
  print("\n")
  break

I want to go to the market to buy fresh fruits and vegetables, affordable spices, and meet friendly locals to learn about their culture and cuisine.
I want to go to the market to buy fresh fruits and vegetables, affordable spices, and meet friendly locals to learn about their culture and cuisine.
I want to go to the market to buy fresh fruits and vegetables, affordable spices, and meet friendly locals to learn about their culture and cuisine.




Stochastic Decoding, controlled by temperature.

In [ ]:
prompt = "In less than 40 words, finish the sentence 'I want to go to the market to"
TEMP = 1.0

while True:
  output = generate_outputs(prompt, TEMP, max_tokens=64, n=3)
  print("\n")
  break

I want to go to the market to buy fresh fruits and vegetables, delicious baked goods, and maybe even find some unique gifts from local vendors. It's a great way to support the community while getting some fresh air and sunshine.
I want to go to the market to buy fresh fruits and vegetables, delicious baked goods, and maybe even find some new recipes to try. It's a chance to discover new foods and support local farmers.
I want to go to the market to buy fresh fruits and vegetables, delicious baked goods, and high-quality meat from local farmers. I also enjoy the hustle and bustle of the market and discovering new products and foods.




In [63]:
few_shot_analysis_examples = """
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.
The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.
The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 7, 24.
A: Adding all the odd numbers (11, 7) gives 18. The answer is True.
The odd numbers in this group add up to an even number: 17, 9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.
"""

In [65]:
question = """
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 1.
A:
"""

In [69]:
cot_template = """
{few_shot_examples}

{question}
Follow the examples as it is and answer the question in the same format as the examples. Do not frame extra sentences.
"""

prompt = PromptTemplate.from_template(cot_template).format(
    few_shot_examples=few_shot_analysis_examples,
    question=question
)

TEMP = 0.1
output = generate_outputs(prompt, TEMP, max_tokens=256, stop_sequences=[".", "\n", ""])

The odd numbers in this group are 15, 5, 13, 1. Adding them up gives 34, which is an even number. So the answer is True.
